### Sponsor / MVNO change for subscribers in DMI

In [1]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
from tqdm import tqdm
from importlib import reload
from os.path import join
import re

import roamability as rb

In [2]:
#######################################################################################
# Define variables here
#######################################################################################

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
out_file = 'toc_sponsor.rcjson'
out_file_mvno = 'toc_mvno.rcjson'

mvno_dict = {'Roamability':'TECHMOB'}
sponsor_dict = {'P4_Combined':'P4_TOTT01', 'Partner_Combined': 'Partner_TOTT01','SMART_Tottol': 'SMART_TOTT01',
               'MB_Tottolli': 'MB_TOTT01', 'S6_ALL': 'S6_TOTT01', 'S8_Tottolli': 'S8_TOTT01'}

# mvno_dict = {'DENT01_COMB':'DENT01_COMB'}
# sponsor_dict = {'P4_DENT01_COMB':'P4_DENT01_COMB', 'Partner_DENT01_C': 'Partner_DENT01_C',
#                 'S6_DENT01_COMB': 'S6_DENT01_COMB', 'MB_DENT01_COMB': 'MB_DENT01_COMB'}

# mvno_dict = {'STI':'STI'}
# sponsor_dict = {'Partner_STI_Comb':'Partner_STI_ALA', 'P4_STI_Comb': 'P4_Block',
#                 'MB_STI': 'MB_STI_ALA', 'S6_STI_Comb': 'S6_BLOCK', 'S6_ONLY_STI': 'S6_STI_ALA'}

# mvno_dict = {'Roamability':'SIMT02'}
# sponsor_dict = {'Partner_SIMT_COM':'Partner_SIMT01', 'P4_SIMTEX_COMB': 'P4_SIMT01_COMB'}

# mvno_dict = {'Roamability':'Freecell'}
# sponsor_dict = {'Partner_All':'Partner_FreeCell', 'MB_ALL': 'MB_FreeCell', 'P4_All': 'P4_FreeCell'}

# mvno_dict = {'Roamability':'TECHMOB'}
# sponsor_dict = {'P4_Combined':'P4_TOTT01', 'Partner_Combined': 'Partner_TOTT01','SMART_Tottol': 'SMART_TOTT01',
#                'MB_Tottolli': 'MB_TOTT01', 'S6_ALL': 'S6_TOTT01', 'S8_Tottolli': 'S8_TOTT01'}

# Ranges
ranges = """
454030227645000-454030227646999,
"""

#######################################################################################

#######################################################################################

# toc cspadmin MoRDVruBVEpXXk6kYQ6D services1

In [3]:
# Define Class

class ImsiRange:

    def __init__(self, ranges, mvno_dict, sponsor_dict):
        self.range_list = self.parse_ranges(ranges)
        self.mvno_dict = mvno_dict
        self.sponsor_dict = sponsor_dict
    
    def parse_ranges(self, ranges):
        """Convert ranges to list"""
        pattern_clean = re.compile('([\n\s])')
        ranges = re.sub(pattern_clean, '', ranges)
        pattern_range = re.compile(r'(\d{15})\D+(\d{15})')
        range_list = []
        matches = pattern_range.finditer(ranges)
        for match in matches:
            range_list.append([match.group(1),match.group(2)])
        return range_list
    
    def prepare_range_list_for_sql(self):
        """Convert ranges in the list to string for SQL"""
        str_range_list_for_sql = ''
        for i, ranges in enumerate(imsi_range.range_list):
            str_range_list_for_sql = str_range_list_for_sql + f'(IMSI BETWEEN {ranges[0]} AND {ranges[1]})'
            if i+1 < len(self):
                str_range_list_for_sql = str_range_list_for_sql + ' OR '
        return str_range_list_for_sql
        
    def check_sponsor(self):
        sql_str = """
SELECT
SUBSTR(s.IMSI,1,12) AS IMSI_RANGE,
SUBSTR(si.IMSI_NUMBER,1,12) AS S_IMSI_RANGE,
sp.NAME AS PROFILE_NAME,
COUNT(*) AS NUM
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND si.SUBSCRIBER_REF IN
(
SELECT RI FROM SUBSCRIBER WHERE
{}
)
GROUP BY SUBSTR(s.IMSI,1,12),SUBSTR(si.IMSI_NUMBER,1,12),sp.NAME
ORDER BY SUBSTR(s.IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_sponsor_gr)
        return df_dmi_sponsor_gr
        
    def get_data_for_sponsor_change(self):
        sql_str = """
SELECT
si.CLI_MSISDN,s.IMSI,si.PROF_ID,si.IMSI_NUMBER,si.flag_confirmed,sp.NAME
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND
({})
ORDER BY s.IMSI,si.PROF_ID""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_sponsor
    
    def prepare_toc_to_cange_sponsor(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_sponsor_change()
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        df_dmi_imsi.NAME.replace(to_replace=self.sponsor_dict, inplace=True)
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.s_imsi.modify,CLI_MSISDN="'
        t2='",FLAG_CONFIRMED="0",IMSI_NUMBER="'
        t3='",PROF_ID="'
        t4='",SPONSOR_REF="'
        t5='",SUBSCRIBER_REF="'
        t6='";'
        for i, [cli, imsi, pid, simsi, sponsor] in enumerate(
            df_dmi_imsi[['CLI_MSISDN','IMSI','PROF_ID','IMSI_NUMBER','NAME']].values):
            ouf.write(t1+str(cli)+t2+str(simsi)+t3+str(pid)+t4+str(sponsor)+t5+str(imsi)+t6+'\n')
        ouf.close()
        
    def check_mvno(self):
        sql_str = """
SELECT
m.NAME AS MVNO_NAME,
SUBSTR(IMSI,1,12) AS IMSI_RANGE,
COUNT(*) AS NUM_SUBS,
MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
FROM SUBSCRIBER s, MVNO m WHERE
m.RI = s.MVNO_REF
AND (
{}
)
GROUP BY m.NAME, SUBSTR(IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_mvno_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_mvno_gr)
        return df_dmi_mvno_gr
        
    def get_data_for_mvno_change(self):
        sql_str = """
SELECT
s.SIM_APP_VER,
m.NAME AS MVNO,
s.IMSI,
s.MSISDN
FROM SUBSCRIBER s, MVNO m
WHERE
s.MVNO_REF = m.RI
AND
({})""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_mvno = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_mvno
    
    def prepare_toc_to_cange_mvno(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_mvno_change()
        display(df_dmi_imsi.groupby('MVNO',as_index=False)['IMSI'].count())
        df_dmi_imsi.MVNO.replace(to_replace=self.mvno_dict, inplace=True)
        display(df_dmi_imsi.groupby('MVNO',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.subscriber.modify,IMSI="'
        t2='",MVNO_REF="'
        t3='";'
        for i, [imsi, mvno] in enumerate(df_dmi_imsi[['IMSI','MVNO']].values):
            ouf.write(t1+str(imsi)+t2+str(mvno)+t3+'\n')
        ouf.close()
        
    def _imsis(self, df):
        for  imsi in df.IMSI:
            yield imsi

    def make_ranges(self, downloads, file_name, print_panges=False):
        df = pd.read_csv(join(downloads, file_name))
        df.sort_values(by='IMSI', inplace=True)

        imsi_num = df.count().IMSI

        ranges_list = []

        gen = self._imsis(df)
        imsi1 = next(gen)

        while True:
            range_list = [imsi1, imsi1, 1]
            try:
                while True:
                    imsi2 = next(gen)
                    if imsi2 - imsi1 == 1:
                        range_list[1] = imsi2
                        imsi1 = imsi2
                    else:
                        range_list[2] = range_list[1] - range_list[0] + 1
                        ranges_list.append(range_list)
                        imsi1 = imsi2
                        break
            except StopIteration:
                break

        range_list[2] = range_list[1] - range_list[0] + 1
        ranges_list.append(range_list)

        df = DataFrame(ranges_list, columns=['IMSI_START', 'IMSI_END', 'NUM'])

        if imsi_num == df.NUM.sum():
            print(f'OK: {imsi_num} subscribers in {len(df)} ranges')
        else:
            print(f'Wrong IMSI count: {imsi_num} in file, {df.NUM.sum()} in ranges')
            
        if print_panges:
            for range_list in df.values:
                print(f'{range_list[0]}-{range_list[1]},')
        
        return df
    
    def __len__(self):
        return len(self.range_list)
    
    def __str__(self):
        """Print ranges"""
        for i, ranges in enumerate(self.range_list):
            print(f'{i+1:02} : {ranges[0]} - {ranges[1]} -> {1+int(ranges[1])-int(ranges[0])}')
        return ''
    
imsi_range = ImsiRange(ranges, mvno_dict, sponsor_dict)

print(imsi_range)

01 : 454030227645000 - 454030227646999 -> 2000



In [9]:
# Prepare ranges from file with IMSIs

input_file = 'Soft_SIMs.csv'

# The input file example:

# IMSI
# 260060149999004
# 260060149999007

df_temp = imsi_range.make_ranges(downloads, input_file, print_panges=False)
df_temp.head()
display(df_temp)
for vals in df_temp.values:
    print(f'{vals[0]}-{vals[1]},')

OK: 83025 subscribers in 11 ranges


,IMSI_START,IMSI_END,NUM
0,260060149910000,260060149934999,25000
1,260060149940000,260060149949999,10000
2,260060149991909,260060149991921,13
3,260060149997000,260060149997999,1000
4,425019613795500,425019613796499,1000
5,425019613930000,425019613944999,15000
6,425019613945001,425019613954999,9999
7,425019613960000,425019613969999,10000
8,425019613998901,425019613998913,13
9,454030227307000,454030227308999,2000


260060149910000-260060149934999,
260060149940000-260060149949999,
260060149991909-260060149991921,
260060149997000-260060149997999,
425019613795500-425019613796499,
425019613930000-425019613944999,
425019613945001-425019613954999,
425019613960000-425019613969999,
425019613998901-425019613998913,
454030227307000-454030227308999,
454030227310000-454030227318999,


In [6]:
# Check MVNO in DMI

df_mvno_temp = imsi_range.check_mvno()

,MVNO_NAME,IMSI_RANGE,NUM_SUBS,MIN_IMSI,MAX_IMSI
0,TECHMOB,454030227645,1000,454030227645000,454030227645999
1,TECHMOB,454030227646,1000,454030227646000,454030227646999


In [5]:
# Prepare TOC commands to change the MVNO in DMI

imsi_range.prepare_toc_to_cange_mvno(downloads, out_file_mvno)

,MVNO,IMSI
0,Roamability,2000


,MVNO,IMSI
0,TECHMOB,2000


,SIM_APP_VER,MVNO,IMSI,MSISDN
1997,255,TECHMOB,454030227645844,8529750135844
1998,255,TECHMOB,454030227645845,8529750135845
1999,255,TECHMOB,454030227645846,8529750135846


In [14]:
# Check Sponsors in DMI for the given ranges

df_sponsor_temp = imsi_range.check_sponsor()

,IMSI_RANGE,S_IMSI_RANGE,PROFILE_NAME,NUM
0,260036611004,260036611004,S6_STI_ALA,500


In [13]:
# Prepare TOC commands to change the Sponsor in DMI

imsi_range.prepare_toc_to_cange_sponsor(downloads, out_file)

,NAME,IMSI
0,S6_ONLY_STI,500


,NAME,IMSI
0,S6_STI_ALA,500


,CLI_MSISDN,IMSI,PROF_ID,IMSI_NUMBER,FLAG_CONFIRMED,NAME
497,485079160997,260036611004997,1,260036611004997, ,S6_STI_ALA
498,485079160998,260036611004998,1,260036611004998, ,S6_STI_ALA
499,485079160999,260036611004999,1,260036611004999, ,S6_STI_ALA


### Developments

In [7]:
# Define Class

class ImsiRange:

    def __init__(self, ranges, mvno_dict, sponsor_dict):
        self.range_list = self.parse_ranges(ranges)
        self.mvno_dict = mvno_dict
        self.sponsor_dict = sponsor_dict
    
    def parse_ranges(self, ranges):
        """Convert ranges to list"""
        pattern_clean = re.compile('([\n\s])')
        ranges = re.sub(pattern_clean, '', ranges)
        pattern_range = re.compile(r'(\d{15})\D+(\d{15})')
        range_list = []
        matches = pattern_range.finditer(ranges)
        for match in matches:
            range_list.append([match.group(1),match.group(2)])
        return range_list
    
    def prepare_range_list_for_sql(self):
        """Convert ranges in the list to string for SQL"""
        str_range_list_for_sql = ''
        for i, ranges in enumerate(imsi_range.range_list):
            str_range_list_for_sql = str_range_list_for_sql + f'(IMSI BETWEEN {ranges[0]} AND {ranges[1]})'
            if i+1 < len(self):
                str_range_list_for_sql = str_range_list_for_sql + ' OR '
        return str_range_list_for_sql
        
    def check_sponsor(self):
        sql_str = """
SELECT
SUBSTR(s.IMSI,1,12) AS IMSI_RANGE,
SUBSTR(si.IMSI_NUMBER,1,12) AS S_IMSI_RANGE,
MIN(si.IMSI_NUMBER) AS MIN_S_IMSI,
MAX(si.IMSI_NUMBER) AS MAX_S_IMSI,
sp.NAME AS PROFILE_NAME,
COUNT(*) AS NUM
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND si.SUBSCRIBER_REF IN
(
SELECT RI FROM SUBSCRIBER WHERE
{}
)
GROUP BY SUBSTR(s.IMSI,1,12),SUBSTR(si.IMSI_NUMBER,1,12),sp.NAME
ORDER BY SUBSTR(s.IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_sponsor_gr)
        return df_dmi_sponsor_gr
        
    def get_data_for_sponsor_change(self):
        sql_str = """
SELECT
si.CLI_MSISDN,s.IMSI,si.PROF_ID,si.IMSI_NUMBER,si.flag_confirmed,sp.NAME
FROM S_IMSI si, SUBSCRIBER s, SPONSOR sp WHERE
si.SUBSCRIBER_REF = s.RI
AND si.SPONSOR_REF = sp.RI
AND
({})
ORDER BY s.IMSI,si.PROF_ID""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_sponsor = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_sponsor
    
    def prepare_toc_to_cange_sponsor(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_sponsor_change()
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        df_dmi_imsi.NAME.replace(to_replace=self.sponsor_dict, inplace=True)
        display(df_dmi_imsi.groupby('NAME',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.s_imsi.modify,CLI_MSISDN="'
        t2='",FLAG_CONFIRMED="0",IMSI_NUMBER="'
        t3='",PROF_ID="'
        t4='",SPONSOR_REF="'
        t5='",SUBSCRIBER_REF="'
        t6='";'
        for i, [cli, imsi, pid, simsi, sponsor] in enumerate(
            df_dmi_imsi[['CLI_MSISDN','IMSI','PROF_ID','IMSI_NUMBER','NAME']].values):
            ouf.write(t1+str(cli)+t2+str(simsi)+t3+str(pid)+t4+str(sponsor)+t5+str(imsi)+t6+'\n')
        ouf.close()
        
    def check_mvno(self):
        sql_str = """
SELECT
m.NAME AS MVNO_NAME,
SUBSTR(IMSI,1,12) AS IMSI_RANGE,
COUNT(*) AS NUM_SUBS,
MIN(IMSI) AS MIN_IMSI,MAX(IMSI) AS MAX_IMSI
FROM SUBSCRIBER s, MVNO m WHERE
m.RI = s.MVNO_REF
AND (
{}
)
GROUP BY m.NAME, SUBSTR(IMSI,1,12)""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_mvno_gr = pd.read_sql_query(sql_str, cnxn, coerce_float=False) 
        display(df_dmi_mvno_gr)
        return df_dmi_mvno_gr
        
    def get_data_for_mvno_change(self):
        sql_str = """
SELECT
s.SIM_APP_VER,
m.NAME AS MVNO,
s.IMSI,
s.MSISDN
FROM SUBSCRIBER s, MVNO m
WHERE
s.MVNO_REF = m.RI
AND
({})""".format(self.prepare_range_list_for_sql())
        with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
            df_dmi_mvno = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
        return df_dmi_mvno
    
    def prepare_toc_to_cange_mvno(self, downloads, out_file):
        df_dmi_imsi = self.get_data_for_mvno_change()
        display(df_dmi_imsi.groupby('MVNO',as_index=False)['IMSI'].count())
        df_dmi_imsi.MVNO.replace(to_replace=self.mvno_dict, inplace=True)
        display(df_dmi_imsi.groupby('MVNO',as_index=False)['IMSI'].count())
        display(df_dmi_imsi.tail(3))
        ouf=open(join(downloads, out_file), 'w')
        t1='dmi.subscriber.modify,IMSI="'
        t2='",MVNO_REF="'
        t3='";'
        for i, [imsi, mvno] in enumerate(df_dmi_imsi[['IMSI','MVNO']].values):
            ouf.write(t1+str(imsi)+t2+str(mvno)+t3+'\n')
        ouf.close()
        
    def _imsis(self, df):
        for  imsi in df.IMSI:
            yield imsi

    def make_ranges(self, downloads, file_name, print_panges=False):
        df = pd.read_csv(join(downloads, file_name))
        df.sort_values(by='IMSI', inplace=True)

        imsi_num = df.count().IMSI

        ranges_list = []

        gen = self._imsis(df)
        imsi1 = next(gen)

        while True:
            range_list = [imsi1, imsi1, 1]
            try:
                while True:
                    imsi2 = next(gen)
                    if imsi2 - imsi1 == 1:
                        range_list[1] = imsi2
                        imsi1 = imsi2
                    else:
                        range_list[2] = range_list[1] - range_list[0] + 1
                        ranges_list.append(range_list)
                        imsi1 = imsi2
                        break
            except StopIteration:
                break

        range_list[2] = range_list[1] - range_list[0] + 1
        ranges_list.append(range_list)

        df = DataFrame(ranges_list, columns=['IMSI_START', 'IMSI_END', 'NUM'])

        if imsi_num == df.NUM.sum():
            print(f'OK: {imsi_num} subscribers in {len(df)} ranges')
        else:
            print(f'Wrong IMSI count: {imsi_num} in file, {df.NUM.sum()} in ranges')
            
        if print_panges:
            for range_list in df.values:
                print(f'{range_list[0]}-{range_list[1]},')
        
        return df
    
    def __len__(self):
        return len(self.range_list)
    
    def __str__(self):
        """Print ranges"""
        for i, ranges in enumerate(self.range_list):
            print(f'{i+1:02} : {ranges[0]} - {ranges[1]} -> {1+int(ranges[1])-int(ranges[0])}')
        return ''
    
imsi_range = ImsiRange(ranges, mvno_dict, sponsor_dict)

print(imsi_range)

01 : 260036611003000 - 260036611004999 -> 2000



In [8]:
imsi_range.check_sponsor()

,IMSI_RANGE,S_IMSI_RANGE,MIN_S_IMSI,MAX_S_IMSI,PROFILE_NAME,NUM
0,260036611003,260036611003,260036611003000,260036611003999,S6_STI_ALA,1000
1,260036611004,260036611004,260036611004500,260036611004999,S6_ONLY_STI,500
2,260036611004,260036611004,260036611004000,260036611004499,S6_STI_ALA,500


,IMSI_RANGE,S_IMSI_RANGE,MIN_S_IMSI,MAX_S_IMSI,PROFILE_NAME,NUM
0,260036611003,260036611003,260036611003000,260036611003999,S6_STI_ALA,1000
1,260036611004,260036611004,260036611004500,260036611004999,S6_ONLY_STI,500
2,260036611004,260036611004,260036611004000,260036611004499,S6_STI_ALA,500
